<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
import numpy as np
import time
import math
import csv

In [2]:
import CMGDB

In [3]:
# Leslie map
def F(x):
    th1 = 19.6
    th2 = 23.68
    return [(th1 * x[0] + th2 * x[1]) * math.exp (-0.1 * (x[0] + x[1])), 0.7 * x[0]]

In the next example we use only a single level of subdivision. For this we create
a model with only one parameter indicating the number of subdivisions. We agian use the
Leslie map defined above.

In [6]:
phase_subdiv = 20
lower_bounds = [-0.001, -0.001]
upper_bounds = [90.0, 70.0]

model = CMGDB.Model(phase_subdiv, lower_bounds, upper_bounds, F)

The function below returns the Morse graph and a digraph representing the
multi-valued map defined on the grid in phase space. The digraph representing
the multi-valued map is computed only on the final grid representing where the
Morse sets are computed, hence it only returns the multi-valued map in the whole
phase space if the model is constructed with a single level of subdivision as
above.

In [7]:
%%time
morse_graph, map_graph = CMGDB.ComputeMorseGraph(model)

CPU times: user 18.9 s, sys: 146 ms, total: 19.1 s
Wall time: 19.1 s


In [51]:
class IteratedGraph():
    def __init__(self,phase_subdiv, lower_bounds, upper_bounds, F=None,map_graph=None,morse_graph=None):
        self.phase_subdiv = phase_subdiv
        self.lower_bounds = lower_bounds
        self.upper_bounds = upper_bounds
        
        if F:
            self.F = F

        self.map_graph = map_graph
        self.morse_graph = morse_graph
        if (not map_graph) or (not morse_graph):
            model = CMGDB.Model(phase_subdiv, lower_bounds, upper_bounds, F)
            self.morse_graph, self.map_graph = CMGDB.ComputeMorseGraph(model)
        self.MORSE_GRAPH = 'morse_graph'
        self.MAP_GRAPH = 'map_graph'
        
    def relabel(self,objs:list):
        '''
        :param objs: a list of objects
        '''
        return [(self.phase_subdiv,item) for item in objs]
        
    def vertices(self,graph_type:str):
        if graph_type == self.MORSE_GRAPH:
            return self.relabel(self.morse_graph.vertices())
        else:
            return self.relabel(self.map_graph.vertices())
        
    def edges(self,graph_type:str):
        if graph_type == self.MORSE_GRAPH:
            return self.relabel(self.morse_graph.edges())
        else:
            return self.relabel(self.map_graph.edges())
        
    def morse_set(self,v_id:int):
        return self.relabel(self.morse_graph(v_id))
    
    def edges_unreduced(self):
        return self.relabel(self.morse_graph.edges_unreduced())
    
    def adjacencies(self,graph_type:str,v_id:int):
        if graph_type == self.MORSE_GRAPH:
            return self.relabel(self.morse_graph.adjacencies(v_id))
        else:
            return self.relabel(self.map_graph.adjacencies(v_id))
        
    def num_vertices(self,graph_type:str):
        if graph_type == self.MORSE_GRAPH:
            return self.morse_graph.num_vertices()
        else:
            return self.map_graph.num_vertices()
        
    def morse_set_boxes(self,v_id:int):
        return self.relabel(self.morse_graph.morse_set_boxes(v_id))
    
    def get_mg_vertex_id(self,ig_vertex:tuple):
        return ig_vertex[1]
        
    
    

In [44]:
ig = IteratedGraph(phase_subdiv,lower_bounds,upper_bounds,morse_graph=morse_graph,map_graph=map_graph)

In [45]:
ig.num_vertices(ig.MORSE_GRAPH)

6

In [46]:
ig.vertices(ig.MORSE_GRAPH)

[(20, 0), (20, 1), (20, 2), (20, 3), (20, 4), (20, 5)]

In [47]:
ig.edges_unreduced()

[(20, (5, 0)),
 (20, (5, 1)),
 (20, (5, 4)),
 (20, (3, 2)),
 (20, (2, 0)),
 (20, (5, 2)),
 (20, (2, 1)),
 (20, (5, 3)),
 (20, (4, 0)),
 (20, (3, 0)),
 (20, (3, 1))]

In [50]:
ig.edges(ig.MORSE_GRAPH)

[(20, (2, 1)),
 (20, (2, 0)),
 (20, (3, 2)),
 (20, (4, 0)),
 (20, (5, 3)),
 (20, (5, 4))]

In [ ]:
class CompositeGraph():
    def __init__(self,ig1:IteratedGraph,ig2:IteratedGraph):
        self.ig1 = ig1
        self.ig2 = ig2
        
    def _get_intervals(self,morse_set_boxes):
        for i,item in enumerate(morse_set_boxes):
            if i == 0:
                mg_x = interval([item[2][0],item[2][2]])
                mg_y = interval([item[2][1],item[2][3]])
                continue
            mg_x = mg_x | interval([item[2][0],item[2][2]])
            mg_y = mg_y | interval([item[2][1],item[2][3]])
        return mg_x,mg_y
    
    
    def forward(self,v:int):
        '''
        Move from coarse to fine
        '''
        if ig1.phase_subdiv < ig2.phase_subdiv:
            print('IG1 is more coarse than IG2. It will be set to graph 1...')
            print()
            graph1 = self.ig1
            graph2 = self.ig2
        else:
            print('IG2 is more coarse than IG1. It will be set to graph 1...')
            print()
            graph1 = self.ig2
            graph2 = self.ig1
        self._traverse(graph1,graph2,v)
        
    def backward(self,v:int):
        '''
        Move from fine to coarse
        '''
        if self.ig1.phase_subdiv > self.ig2.phase_subdiv:
            print('IG1 is more coarse than IG2. It will be set to graph 1...')
            print()
            graph1 = self.ig1
            graph2 = self.ig2
        else:
            print('IG2 is more coarse than IG1. It will be set to graph 1...')
            print()
            graph1 = self.ig2
            graph2 = self.ig1
            
    #Returns true if two rectangles (l1, r1) and (l2, r2) overlap 
    def _doOverlap(self,box1,box2):
        l1x,l1y,r1x,r1y = box1
        l2x,l2y,r2x,r2y = box2
        # If one rectangle is on left side of other 
        if (l1x >= r2x) | (l2x >= r1x) 
            return false 

        #If one rectangle is above other 
        if (l1y <= r2y) | (l2y <= r1y) 
            return false 

        return true 
   
    def _traverse(self,graph1,graph2,v):
        '''
        A traversal will return all graph 2 nodes that v in graph 1 connects to
        '''
        morse_set_boxes_1 = graph1.morse_set_boxes(v)
        mg1_x,mg1_y = self._get_intervals(morse_set_boxes_1)
        
        # Check if graph2 nodes are in the interval of graph 1. If they are, return those nodes:
        vertices = []
        for item in graph2.vertices():
            morse_set_boxes_2 = graph2.morse_set_boxes(item[2])
            mg2_x,mg2_y = self._get_intervals(morse_set_boxes_2)
            

        
            
        
            
        